# Importing essential Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
#!conda install -c conda-forge folium --yes
import folium
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
# The code was removed by Watson Studio for sharing.

,Unnamed: 0,Borough,Neighborhood,latitude,longitude
0,0,North West Delhi,Adarsh Nagar,28.614192,77.071541
1,1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,2,North West Delhi,Azadpur,28.707657,77.175547
3,3,North West Delhi,Bawana,28.799660,77.032885
4,4,North West Delhi,Begum Pur,NaN,NaN


In [3]:
delhi_data.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
delhi_data.head()

,Borough,Neighborhood,latitude,longitude
0,North West Delhi,Adarsh Nagar,28.614192,77.071541
1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,North West Delhi,Azadpur,28.707657,77.175547
3,North West Delhi,Bawana,28.799660,77.032885
4,North West Delhi,Begum Pur,NaN,NaN


In [4]:
delhi_data.isnull().sum()

Borough          0
Neighborhood     0
latitude        22
longitude       22
dtype: int64

In [5]:
delhi_data.dropna(axis=0,inplace=True)

In [6]:
delhi_data.shape

(163, 4)

In [7]:
address='Delhi, India'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Delhi, India are 28.6517178, 77.2219388.


In [8]:
map_delhi=folium.Map(location=[latitude,longitude],zoom_start=12)
for lat,lng,label in zip(delhi_data['latitude'],delhi_data['longitude'],delhi_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_delhi)
map_delhi

In [9]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: 0P03LNSYETL5NOO2FVW1RD2QUN4A020XF2H4QCHIAEE2UX43
CLIENT_SECRET:EMFHNSIQJTIPIVSGYM31NOQSQQUMPRO3I4TI00NLRDPQJ4UD


In [10]:
LIMIT = 100
radius = 2000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=0P03LNSYETL5NOO2FVW1RD2QUN4A020XF2H4QCHIAEE2UX43&client_secret=EMFHNSIQJTIPIVSGYM31NOQSQQUMPRO3I4TI00NLRDPQJ4UD&v=20200202&ll=28.6517178,77.2219388&radius=2000&limit=100'

In [11]:
results = requests.get(url).json()

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
delhi_venues = getNearbyVenues(names=delhi_data['Neighborhood'],
                                   latitudes=delhi_data['latitude'],
                                   longitudes=delhi_data['longitude']
                                  )

Adarsh Nagar
Ashok Vihar
Azadpur
Bawana
Dhaka
Jahangirpuri
Karala
Keshav Puram
Kingsway Camp
Kohat Enclave
Model Town
Narela
Pitam Pura
Rani Bagh
Rithala
Shalimar Bagh
Shakti Nagar
Bara Hindu Rao
Chandni Chowk
Chawri Bazaar
Civil Lines
Dariba Kalan
Kamla Nagar
Kashmiri Gate
Kotwali
Lahori Gate
Nehru Vihar
Paharganj
Pratap Nagar
Pul Bangash
Roshanara Bagh
Sangam Vihar
Sant Nagar
Sarai Kale Khan
Sarai Rohilla
Shakti Nagar
Shastri Nagar
Timarpur
Wazirabad
Babarpur
Dilshad Garden
Kabir Nagar
Mandoli
Maujpur
Nand Nagri
Naveen Shahdara
Shahdara
Sonia Vihar
Shastri Park
Yamuna Vihar
Chandni Chowk
Daryaganj
Jhandewalan
Karol Bagh
Bara Hindu Rao
Paharganj
Barakhamba Road
Chanakyapuri
Connaught Place
Gautampuri
Gole Market
Golf Links
INA Colony
Khan Market
Pragati Maidan
Brij Puri
East Vinod Nagar
Gandhi Nagar
Geeta Colony
Jhilmil Colony
Krishna Nagar
Laxmi Nagar
Mayur Vihar
Pandav Nagar
Patparganj
Preet Vihar
Anand Vihar
Shakarpur
Shastri Nagar
Vasundhara Enclave
Vishwas Nagar
Vivek Vihar
Shahd

In [14]:
print(delhi_venues.shape)

print("\n\n")

delhi_venues.head()

(1070, 7)





,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adarsh Nagar,28.614192,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant
1,Ashok Vihar,28.699453,77.184826,Nat Khat Caterers,28.699630,77.187832,Indian Restaurant
2,Ashok Vihar,28.699453,77.184826,ELIXIR FERTILITY CENTRE,28.700034,77.188093,Health & Beauty Service
3,Ashok Vihar,28.699453,77.184826,Bakers Stop,28.700495,77.188716,Bakery
4,Ashok Vihar,28.699453,77.184826,Invitation Banquet,28.696018,77.185953,Diner


In [15]:
delhi_venues.groupby(['Neighborhood']).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adarsh Nagar,1,1,1,1,1,1
Alaknanda,10,10,10,10,10,10
Anand Vihar,8,8,8,8,8,8
Ashok Vihar,5,5,5,5,5,5
Azadpur,3,3,3,3,3,3
Babarpur,4,4,4,4,4,4
Badarpur,4,4,4,4,4,4
Bali Nagar,6,6,6,6,6,6
Bara Hindu Rao,8,8,8,8,8,8


In [16]:
#one hot encoding

delhi_venues_one_hot=pd.get_dummies(delhi_venues[['Venue Category']], prefix="", prefix_sep="")

delhi_venues_one_hot['Neighborhood'] = delhi_venues['Neighborhood'] 

fixed_columns = [delhi_venues_one_hot.columns[-1]] + list(delhi_venues_one_hot.columns[:-1])
delhi_venues_one_hot = delhi_venues_one_hot[fixed_columns]

#print the shape of the dataframe
print(delhi_venues_one_hot.shape)

#display first 5 rows of the dataframe
delhi_venues_one_hot.head()


(1070, 168)


,Women's Store,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Bus Station,Business Service,Café,Campground,Castle,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Fabric Shop,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hockey Field,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Mosque,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multiplex,Museum,Music Venue,Neighborhood,Night Market,Nightclub,Nightlife Spot,North Indian Restaurant,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pizza Place,Platform,Playground,Plaza,Pool Hall,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Restaurant,Road,Rock Club,Sake Bar,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashok Vihar,0,0,0,0,0,0,0

In [17]:
venues_grouped = delhi_venues_one_hot.groupby(['Neighborhood']).mean().reset_index()

print(venues_grouped.shape)
venues_grouped

(137, 168)


,Neighborhood,Women's Store,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Bus Station,Business Service,Café,Campground,Castle,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Fabric Shop,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hockey Field,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Mosque,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multiplex,Museum,Music Venue,Night Market,Nightclub,Nightlife Spot,North Indian Restaurant,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pizza Place,Platform,Playground,Plaza,Pool Hall,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Restaurant,Road,Rock Club,Sake Bar,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar
0,Adarsh Nagar,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0000,0.00,0.00,0.00,0.00,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00
1,Alaknanda,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.10000

In [18]:
venues_restaurant = venues_grouped[['Neighborhood','Afghan Restaurant','American Restaurant','Asian Restaurant','Australian Restaurant','Chinese Restaurant',
                                   'Eastern European Restaurant','French Restaurant','Indian Restaurant','Italian Restaurant','Japanese Restaurant',
                                   'Mexican Restaurant']]
venues_restaurant.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Australian Restaurant,Chinese Restaurant,Eastern European Restaurant,French Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Mexican Restaurant
0,Adarsh Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,0.0,0.0
1,Alaknanda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.20,0.0,0.0,0.0
2,Anand Vihar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0
3,Ashok Vihar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.40,0.0,0.0,0.0
4,Azadpur,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [19]:
sum_column = venues_restaurant['Afghan Restaurant']+ venues_restaurant['American Restaurant']+venues_restaurant['Asian Restaurant']+\
             venues_restaurant['Australian Restaurant']+venues_restaurant['Chinese Restaurant']+venues_restaurant['Eastern European Restaurant']+\
             venues_restaurant['French Restaurant']+venues_restaurant['Indian Restaurant']+venues_restaurant['Italian Restaurant']+\
             venues_restaurant['Japanese Restaurant']+venues_restaurant['Mexican Restaurant']

In [20]:
venues_restaurant['Restaurant']=sum_column
venues_restaurant.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Australian Restaurant,Chinese Restaurant,Eastern European Restaurant,French Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Restaurant
0,Adarsh Nagar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,0.0,0.0,1.00
1,Alaknanda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.20,0.0,0.0,0.0,0.20
2,Anand Vihar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.25
3,Ashok Vihar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.40,0.0,0.0,0.0,0.40
4,Azadpur,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00


In [21]:
from sklearn.cluster import KMeans

kclusters = 5

res_clustering = venues_restaurant.drop(['Neighborhood','Afghan Restaurant','American Restaurant','Asian Restaurant','Australian Restaurant','Chinese Restaurant',
                                   'Eastern European Restaurant','French Restaurant','Indian Restaurant','Italian Restaurant','Japanese Restaurant',
                                   'Mexican Restaurant'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(res_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([1, 3, 4, 2, 0, 0, 0, 4, 2, 3, 0, 0, 0, 0, 4, 2, 0, 2, 4, 4],
      dtype=int32)

In [22]:
res_merged = venues_restaurant.copy()

res_merged = venues_restaurant.drop(['Afghan Restaurant','American Restaurant','Asian Restaurant','Australian Restaurant','Chinese Restaurant',
                                   'Eastern European Restaurant','French Restaurant','Indian Restaurant','Italian Restaurant','Japanese Restaurant',
                                   'Mexican Restaurant'], 1)

# add clustering labels
res_merged["Cluster Labels"] = kmeans.labels_

In [23]:
res_merged = res_merged.join(delhi_data.set_index("Neighborhood"), on="Neighborhood")

print(res_merged.shape)
res_merged.head()

(145, 6)


,Neighborhood,Restaurant,Cluster Labels,Borough,latitude,longitude
0,Adarsh Nagar,1.00,1,North West Delhi,28.614192,77.071541
1,Alaknanda,0.20,3,South Delhi,28.529336,77.251632
2,Anand Vihar,0.25,4,East Delhi,28.625691,77.101941
3,Ashok Vihar,0.40,2,North West Delhi,28.699453,77.184826
4,Azadpur,0.00,0,North West Delhi,28.707657,77.175547


In [24]:
print(res_merged.shape)
res_merged.sort_values(["Cluster Labels"], inplace=True)
res_merged

(145, 6)


,Neighborhood,Restaurant,Cluster Labels,Borough,latitude,longitude
136,Yamuna Vihar,0.000000,0,North East Delhi,28.700372,77.272773
56,Kingsway Camp,0.000000,0,North West Delhi,28.614262,77.201555
90,Palam,0.000000,0,South West Delhi,28.567173,77.134032
92,Paschim Vihar,0.000000,0,West Delhi,28.669578,77.095956
93,Patel Nagar,0.000000,0,West Delhi,28.659809,77.156957
52,Keshav Puram,0.000000,0,North West Delhi,28.688926,77.161683
96,Pragati Maidan,0.000000,0,New Delhi,28.623459,77.242512
50,Karol Bagh,0.000000,0,Central Delhi,28.652998,77.189023
97,Pratap Nagar,0.000000,0,North Delhi,28.666718,77.198897
99,Punjabi Bagh,0.000000,0,West Delhi,28.668945,77.132461


In [25]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, zipcode, cluster in zip(res_merged['latitude'], res_merged['longitude'], res_merged['Neighborhood'], res_merged['Cluster Labels']):
    label = folium.Popup(str(zipcode) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [26]:
res_merged.loc[res_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Borough,latitude,longitude
136,Yamuna Vihar,0.000000,0,North East Delhi,28.700372,77.272773
56,Kingsway Camp,0.000000,0,North West Delhi,28.614262,77.201555
90,Palam,0.000000,0,South West Delhi,28.567173,77.134032
92,Paschim Vihar,0.000000,0,West Delhi,28.669578,77.095956
93,Patel Nagar,0.000000,0,West Delhi,28.659809,77.156957
52,Keshav Puram,0.000000,0,North West Delhi,28.688926,77.161683
96,Pragati Maidan,0.000000,0,New Delhi,28.623459,77.242512
50,Karol Bagh,0.000000,0,Central Delhi,28.652998,77.189023
97,Pratap Nagar,0.000000,0,North Delhi,28.666718,77.198897
99,Punjabi Bagh,0.000000,0,West Delhi,28.668945,77.132461


In [27]:
res_merged.loc[res_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Borough,latitude,longitude
0,Adarsh Nagar,1.0,1,North West Delhi,28.614192,77.071541
59,Kishangarh Village,1.0,1,South West Delhi,28.519428,77.166535


In [28]:
res_merged.loc[res_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Borough,latitude,longitude
54,Khanpur,0.500000,2,South Delhi,28.512798,77.232395
95,Pitam Pura,0.400000,2,North West Delhi,28.703268,77.132250
94,Patparganj,0.428571,2,East Delhi,28.611592,77.290564
117,Shahpur Jat,0.428571,2,South Delhi,28.548330,77.214104
108,Saket,0.454545,2,South Delhi,28.524411,77.213725
17,Civil Lines,0.500000,2,North Delhi,28.676857,77.225056
15,Chhattarpur,0.400000,2,South Delhi,28.507007,77.175417
45,Jhilmil Colony,0.500000,2,East Delhi,28.669814,77.307283
88,Okhla,0.400000,2,South Delhi,28.563662,77.289055
8,Bara Hindu Rao,0.500000,2,Central Delhi,28.659518,77.205010


In [29]:
res_merged.loc[res_merged['Cluster Labels'] == 3]

,Neighborhood,Restaurant,Cluster Labels,Borough,latitude,longitude
76,Munirka,0.090909,3,South West Delhi,28.554886,77.171084
31,Gole Market,0.200000,3,New Delhi,28.633719,77.205627
30,Ghitorni,0.142857,3,South West Delhi,28.493811,77.149166
68,Mahipalpur,0.125000,3,South West Delhi,28.544485,77.125691
115,Sarvodaya Enclave,0.200000,3,South Delhi,28.537478,77.202089
9,Barakhamba Road,0.142857,3,New Delhi,28.629142,77.226149
128,Timarpur,0.200000,3,North Delhi,28.701263,77.218680
131,Vasant Vihar,0.210526,3,South West Delhi,28.560691,77.160791
1,Alaknanda,0.200000,3,South Delhi,28.529336,77.251632
26,East of Kailash,0.090909,3,South Delhi,28.557032,77.244614


In [30]:
res_merged.loc[res_merged['Cluster Labels'] == 4]

,Neighborhood,Restaurant,Cluster Labels,Borough,latitude,longitude
125,South Extension,0.307692,4,South Delhi,28.568566,77.220568
87,Nizamuddin West,0.250000,4,South Delhi,28.588365,77.244955
61,Kotwali,0.294118,4,North Delhi,28.658598,77.231084
37,Hauz Khas Village,0.274510,4,South Delhi,28.553855,77.194713
7,Bali Nagar,0.333333,4,West Delhi,28.654138,77.128178
2,Anand Vihar,0.250000,4,East Delhi,28.625691,77.101941
133,Vikaspuri,0.333333,4,West Delhi,28.638419,77.070836
66,Lodi Colony,0.285714,4,South Delhi,28.590702,77.220921
62,Krishna Nagar,0.250000,4,East Delhi,28.563702,77.193410
14,Chawri Bazaar,0.300000,4,North Delhi,28.649927,77.229788
